# Agricultural Commodity Supply and Pricing Analysis in Kenya: Data Cleaning
#### Author: Edward Njiru
#### Date: September, 2024

## Introduction
This notebook focuses on cleaning and preparing a dataset that contains information on the supply and pricing of various agricultural commodities in Kenya. The dataset is a combination of seasonal crops, perennial crops, livestock, fresh and saltwater fish, and agricultural inputs.

### Objectives
- Handle missing values
- Correct data types for price and date columns
- Remove duplicates
- Handle mixed data types
- Prepare the dataset for exploratory data analysis (EDA) and future predictive modeling

---

### Data Inspection


In [33]:
# Import necessary libraries
import pandas as pd
import numpy as np
import warnings

# Ignore warnings for cleaner output
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

# Load the dataset
file_path = r"H:\Datasets\Agriculture Kenya\Food Prices\KAMIS 2009-2024\Combined dataset\combined_datasets.csv"
combined_data = pd.read_csv(file_path)

# Display the first few rows to understand the structure
combined_data.head()


,commodity,classification,grade,market,wholesale_price,retail_price,volume_supplied,county,date
0,African butter catfish,Dried,-,Kipini Fish market,250.0,330.0,62.0,Tana-River,17/10/2023
1,African butter catfish,Fresh,-,Kipini Fish market,80.0,150.0,17.0,Tana-River,26/09/2023
2,African butter catfish,Fresh,-,Kipini Fish market,100.0,150.0,25.0,Tana-River,19/09/2023
3,African butter catfish,Fresh,-,Kipini Fish market,100.0,160.0,28.0,Tana-River,13/09/2023
4,African butter catfish,Fresh,-,Kipini Fish market,100.0,160.0,28.0,Tana-River,06/09/2023


In [34]:
# Basic dataset info and summary statistics
combined_data.info()
combined_data.describe(include='all')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025569 entries, 0 to 1025568
Data columns (total 9 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   commodity        1022563 non-null  object
 1   classification   1019497 non-null  object
 2   grade            1022563 non-null  object
 3   market           1022563 non-null  object
 4   wholesale_price  887397 non-null   object
 5   retail_price     670271 non-null   object
 6   volume_supplied  690755 non-null   object
 7   county           1022563 non-null  object
 8   date             1022563 non-null  object
dtypes: object(9)
memory usage: 70.4+ MB


,commodity,classification,grade,market,wholesale_price,retail_price,volume_supplied,county,date
count,1022563,1019497,1022563,1022563,887397.0,670271.0,690755.0,1022563,1022563
unique,176,149,16,301,9000.0,8685.0,5049.0,49,2078
top,Cattle,-,-,Nakuru Wakulima,50.0,100.0,200.0,Nairobi,24/05/2021
freq,39540,693406,916032,66486,44574.0,65104.0,23779.0,95561,70173


In [35]:
# Check the data types and non-null counts of each column
combined_data.info()

# --- Check for Missing Values ---
missing_values = combined_data.isnull().sum()
print("\nMissing Values per Column:")
print(missing_values)

# --- Check Data Types for Each Column ---
data_types = combined_data.dtypes
print("\nData Types for Each Column:")
print(data_types)

# --- Check for Fully Blank Rows ---
fully_blank_rows = combined_data[combined_data.isnull().all(axis=1)]
fully_blank_count = fully_blank_rows.shape[0]
print(f"\nNumber of Fully Blank Rows: {fully_blank_count}")

# Display preview if any fully blank rows exist
if fully_blank_count > 0:
    print(f"\nPreview of Fully Blank Rows:\n{fully_blank_rows.head()}")

# --- Check for Duplicate Rows ---
duplicate_count = combined_data.duplicated().sum()
print(f"\nNumber of Duplicate Rows: {duplicate_count}")

# --- View Summary Statistics of the Numerical Columns ---
print("\nSummary Statistics for Numerical Columns:")
print(combined_data.describe())

# --- View Unique Values in 'commodity' Column to Understand Categories ---
unique_commodities = combined_data['commodity'].unique()
print(f"\nUnique Commodities: {len(unique_commodities)}\n{unique_commodities}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025569 entries, 0 to 1025568
Data columns (total 9 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   commodity        1022563 non-null  object
 1   classification   1019497 non-null  object
 2   grade            1022563 non-null  object
 3   market           1022563 non-null  object
 4   wholesale_price  887397 non-null   object
 5   retail_price     670271 non-null   object
 6   volume_supplied  690755 non-null   object
 7   county           1022563 non-null  object
 8   date             1022563 non-null  object
dtypes: object(9)
memory usage: 70.4+ MB

Missing Values per Column:
commodity            3006
classification       6072
grade                3006
market               3006
wholesale_price    138172
retail_price       355298
volume_supplied    334814
county               3006
date                 3006
dtype: int64

Data Types for Each Column:
commodity          object


### Data Inspection Findings

1. **Dataset Overview**:
   - **Total Rows**: 1,025,569
   - **Total Columns**: 9
   - **Memory Usage**: 70.4 MB
   - All columns are currently stored as `object` type, even those that should be numeric (prices and volume). The `date` column is stored as a string and needs to be converted to a proper `datetime` format.

2. **Missing Values**:
   - **commodity**: 3,006 missing values.
   - **classification**: 6,072 missing values.
   - **grade**: 3,006 missing values.
   - **market**: 3,006 missing values.
   - **wholesale_price**: 138,172 missing values.
   - **retail_price**: 355,298 missing values.
   - **volume_supplied**: 334,814 missing values.
   - **county**: 3,006 missing values.
   - **date**: 3,006 missing values.

3. **Fully Blank Rows**:
   - There are 3,006 fully blank rows (where all columns are NaN). These rows provide no valuable information and should be removed.

4. **Duplicate Rows**:
   - The dataset contains **42,533 duplicate rows**, which should be removed to prevent skewing analysis.

5. **Summary Statistics**:
   - **Commodity**: 176 unique values, with Cattle being the most frequent (39,540 occurrences).
   - **Market**: 301 unique market locations.
   - **Wholesale Price**: 9,000 unique values, most frequent being 50 KSH (44,574 occurrences).
   - **Retail Price**: 8,685 unique values, most common being 100 KSH (65,104 occurrences).
   - **Volume Supplied**: 5,049 unique values, with 200 kg as the most frequent volume supplied.
   - **County**: 49 unique counties, with Nairobi being the most common.

6. **Commodities**:
   - There are 177 unique commodity entries, ranging from crops to livestock, fish, and other goods. Some entries contain NaN values that should be addressed.


---

### Data Cleaning

In this section, I will clean the dataset by:
1. Removing fully blank rows and duplicates.
2. Renaming columns for clarity.
3. Handling missing values (imputation and filling).
4. Converting data types.
5. Calculating resell profit.
6. Rearranging columns for better organization.


In [36]:
# Step 1: Delete Fully Blank Rows
# Remove rows where all column values are NaN, as these rows do not provide any valuable information.
combined_data.dropna(how='all', inplace=True)

# Step 2: Remove Duplicate Rows
# Drop any duplicate rows in the dataset to ensure data integrity.
combined_data.drop_duplicates(inplace=True)

# Check the result after handling blank and duplicate rows
print("Number of rows after removing fully blank and duplicate rows:", combined_data.shape[0])


Number of rows after removing fully blank and duplicate rows: 983035


In [37]:
# Step 3: Rename Columns
# Rename columns for consistency and better understanding. This makes the column names more readable.
combined_data.rename(columns={
    'wholesale_price': 'wholesale_price_ksh',
    'retail_price': 'retail_price_ksh',
    'volume_supplied': 'volume_supplied_kg'
}, inplace=True)

# Display the first few rows to verify changes
combined_data.head()


,commodity,classification,grade,market,wholesale_price_ksh,retail_price_ksh,volume_supplied_kg,county,date
0,African butter catfish,Dried,-,Kipini Fish market,250.0,330.0,62.0,Tana-River,17/10/2023
1,African butter catfish,Fresh,-,Kipini Fish market,80.0,150.0,17.0,Tana-River,26/09/2023
2,African butter catfish,Fresh,-,Kipini Fish market,100.0,150.0,25.0,Tana-River,19/09/2023
3,African butter catfish,Fresh,-,Kipini Fish market,100.0,160.0,28.0,Tana-River,13/09/2023
4,African butter catfish,Fresh,-,Kipini Fish market,100.0,160.0,28.0,Tana-River,06/09/2023


In [38]:
# Step 5: Handle Missing Price, Volume, and Categorical Values
# Convert columns to numeric first, impute missing values using the mean between the previous and next valid values,
# then apply forward fill to fill any remaining gaps, and finally handle categorical missing values.

# Convert price and volume columns to numeric (coerce any invalid values to NaN)
combined_data['wholesale_price_ksh'] = pd.to_numeric(combined_data['wholesale_price_ksh'], errors='coerce')
combined_data['retail_price_ksh'] = pd.to_numeric(combined_data['retail_price_ksh'], errors='coerce')
combined_data['volume_supplied_kg'] = pd.to_numeric(combined_data['volume_supplied_kg'], errors='coerce')

# Function to impute missing values by calculating the mean between the previous and next valid values
def impute_mean_between(data_series):
    nan_indices = data_series[data_series.isna()].index
    for i in nan_indices:
        prev_value = data_series[:i].last_valid_index()
        next_value = data_series[i:].first_valid_index()
        if prev_value is not None and next_value is not None:
            mean_value = (data_series[prev_value] + data_series[next_value]) / 2
            data_series.at[i] = mean_value
    return data_series

# Apply the imputation function to wholesale_price_ksh, retail_price_ksh, and volume_supplied_kg
combined_data['wholesale_price_ksh'] = impute_mean_between(combined_data['wholesale_price_ksh'])
combined_data['retail_price_ksh'] = impute_mean_between(combined_data['retail_price_ksh'])
combined_data['volume_supplied_kg'] = impute_mean_between(combined_data['volume_supplied_kg'])

# Apply forward fill for any remaining missing values
combined_data['wholesale_price_ksh'] = combined_data['wholesale_price_ksh'].ffill()
combined_data['retail_price_ksh'] = combined_data['retail_price_ksh'].ffill()
combined_data['volume_supplied_kg'] = combined_data['volume_supplied_kg'].ffill()

# Handle missing values in 'classification' column by filling with 'Regular'
combined_data['classification'].fillna('Regular', inplace=True)

# Display the first few rows after handling missing values
combined_data.head()

# Check remaining missing values to ensure everything is filled correctly
print("\nRemaining Missing Values per Column:")
print(combined_data.isnull().sum())



Remaining Missing Values per Column:
commodity              0
classification         0
grade                  0
market                 0
wholesale_price_ksh    0
retail_price_ksh       0
volume_supplied_kg     0
county                 0
date                   0
dtype: int64


### Dropping Unnecessary Columns

In this step, we will remove rows where the `market` column has missing values, as `market` is an essential feature for our analysis. Having missing data in this column could hinder accurate market-level insights.

Additionally, we will drop the `grade` column because it contains many insignificant values and overlaps with the `classification` column. The `classification` column already provides sufficient differentiation for our analysis, making `grade` redundant.

This will streamline the dataset and remove unnecessary complexity, allowing for a more focused analysis moving forward.


In [39]:
# Step 6: Drop rows where 'market' column has missing values
combined_data.dropna(subset=['market'], inplace=True)

# Drop the 'grade' column as it overlaps with 'classification' and contains many insignificant values
combined_data.drop(columns=['grade'], inplace=True)

# Display the first few rows to confirm the changes
combined_data.head()

# Check if there are any remaining missing values after dropping the 'market' column
print("\nRemaining Missing Values per Column After Dropping 'market' and 'grade':")
print(combined_data.isnull().sum())



Remaining Missing Values per Column After Dropping 'market' and 'grade':
commodity              0
classification         0
market                 0
wholesale_price_ksh    0
retail_price_ksh       0
volume_supplied_kg     0
county                 0
date                   0
dtype: int64


### Setting the Date Column as Index and Rearranging Columns

In this step, I will set the `date` column as the index of the dataset to enable time-series operations. Time is a crucial factor for analysis in this project, especially when analyzing trends in commodity pricing and supply over time.

Additionally, I will rearrange the remaining columns in a logical order, with `commodity`, `classification`, and `county` appearing first, followed by the price and volume columns.

This rearrangement will streamline the dataset, making it easier to work with and analyze.


In [40]:
# Step 7: Set the 'date' column as the index for time-series analysis
combined_data['date'] = pd.to_datetime(combined_data['date'], errors='coerce')  # Ensure 'date' is in datetime format
combined_data.set_index('date', inplace=True)

# Rearrange the columns in a logical order
columns_order = ['commodity', 'classification', 'county', 'market', 
                 'volume_supplied_kg', 'wholesale_price_ksh', 'retail_price_ksh']

# Apply the new column order
combined_data = combined_data[columns_order]

# Display the updated dataframe to confirm changes
combined_data.head()

# Check the index to ensure 'date' is correctly set
print("\nIndex Type and Sample Data:")
print(combined_data.index)


C:\Users\Pavilion\AppData\Local\Temp\ipykernel_9884\885785799.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  combined_data['date'] = pd.to_datetime(combined_data['date'], errors='coerce')  # Ensure 'date' is in datetime format



Index Type and Sample Data:
DatetimeIndex(['2023-10-17', '2023-09-26', '2023-09-19', '2023-09-13',
               '2023-09-06', '2023-09-05', '2023-09-04', '2023-08-15',
               '2023-08-14', '2023-08-07',
               ...
               '2022-10-24', '2022-09-08', '2021-11-04', '2021-10-13',
               '2021-05-24', '2021-05-24', '2021-05-24', '2021-05-24',
               '2021-05-24', '2021-05-24'],
              dtype='datetime64[ns]', name='date', length=983035, freq=None)


### Converting Volume to Tonnes

In this step, I will convert the `volume_supplied_kg` column to tonnes, as 1 tonne equals 1,000 kg. I will then rename the column to `volume_supplied_tonnes` for clarity and drop the original `volume_supplied_kg` column, which is no longer needed.

This conversion allows for easier interpretation and comparison of commodity volumes, especially when dealing with larger quantities.


In [41]:
# Step 8: Convert 'volume_supplied_kg' to 'volume_supplied_tonnes'
combined_data['volume_supplied_tonnes'] = combined_data['volume_supplied_kg'] / 1000

# Drop the original 'volume_supplied_kg' column if it's no longer needed
combined_data.drop(columns=['volume_supplied_kg'], inplace=True)

# Display the first few rows to confirm the changes
combined_data.head()


,commodity,classification,county,market,wholesale_price_ksh,retail_price_ksh,volume_supplied_tonnes
date,,,,,,,
2023-10-17,African butter catfish,Dried,Tana-River,Kipini Fish market,250.0,330.0,0.062
2023-09-26,African butter catfish,Fresh,Tana-River,Kipini Fish market,80.0,150.0,0.017
2023-09-19,African butter catfish,Fresh,Tana-River,Kipini Fish market,100.0,150.0,0.025
2023-09-13,African butter catfish,Fresh,Tana-River,Kipini Fish market,100.0,160.0,0.028
2023-09-06,African butter catfish,Fresh,Tana-River,Kipini Fish market,100.0,160.0,0.028


In [42]:
# Rearrange the columns to place 'volume_supplied_tonnes' after 'market'
columns_order = ['commodity', 'classification', 'county', 'market', 
                 'volume_supplied_tonnes', 'wholesale_price_ksh', 'retail_price_ksh']

# Apply the new column order
combined_data = combined_data[columns_order]

# Display the updated dataframe to confirm the changes
combined_data.head()


,commodity,classification,county,market,volume_supplied_tonnes,wholesale_price_ksh,retail_price_ksh
date,,,,,,,
2023-10-17,African butter catfish,Dried,Tana-River,Kipini Fish market,0.062,250.0,330.0
2023-09-26,African butter catfish,Fresh,Tana-River,Kipini Fish market,0.017,80.0,150.0
2023-09-19,African butter catfish,Fresh,Tana-River,Kipini Fish market,0.025,100.0,150.0
2023-09-13,African butter catfish,Fresh,Tana-River,Kipini Fish market,0.028,100.0,160.0
2023-09-06,African butter catfish,Fresh,Tana-River,Kipini Fish market,0.028,100.0,160.0


### Re-inspecting the DataFrame

Now that the volume has been converted, I will re-inspect the dataset to ensure everything looks correct. This includes checking the basic structure of the DataFrame, such as data types, missing values, and summary statistics for numerical columns like prices and volume.

These checks will give me a preliminary understanding of the data distributions before proceeding with visualizations.


In [43]:
# Step 9: Re-inspect the data and check basic statistics
print("\nData Summary:")
combined_data.info()

print("\nSummary Statistics for Numerical Columns:")
print(combined_data.describe())



Data Summary:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 983035 entries, 2023-10-17 to 2021-05-24
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   commodity               983035 non-null  object 
 1   classification          983035 non-null  object 
 2   county                  983035 non-null  object 
 3   market                  983035 non-null  object 
 4   volume_supplied_tonnes  983035 non-null  float64
 5   wholesale_price_ksh     983035 non-null  float64
 6   retail_price_ksh        983035 non-null  float64
dtypes: float64(3), object(4)
memory usage: 60.0+ MB

Summary Statistics for Numerical Columns:
       volume_supplied_tonnes  wholesale_price_ksh  retail_price_ksh
count           983035.000000         9.830350e+05      9.830350e+05
mean                 5.329091         2.544467e+03      2.898955e+03
std                359.856038         4.920758e+04      1.519309e+04
m

### Identifying Issues in the Summary Statistics

Upon inspecting the summary statistics for the numerical columns (`volume_supplied_tonnes`, `wholesale_price_ksh`, and `retail_price_ksh`), a few inconsistencies stand out that require further investigation:

1. **Minimum Values of 0**: 
   - The `volume_supplied_tonnes`, `wholesale_price_ksh`, and `retail_price_ksh` columns all have minimum values of 0. This is problematic because:
     - For **volume**: A value of 0 suggests that no commodity was supplied to the market, which contradicts the expectation that the data records only active sales and market supplies.
     - For **prices**: A value of 0 for both wholesale and retail prices is illogical, as it implies that commodities were traded for free, which is highly unlikely in a market setting.
   
   These entries need to be investigated further to determine whether they are errors, missing data that were improperly handled, or actual records with special meanings.

2. **Extreme Maximum Values**: 
   - The maximum value for **wholesale price** is over 15 million KSH, and for **retail price**, it is 4 million KSH. These values are extremely high and may represent data entry errors or outliers.
   
   I will investigate these extreme values and decide on the appropriate treatment, such as capping or removing them if they are deemed outliers.

3. **Distribution of Data**: 
   - After addressing the zero values and extreme outliers, I will inspect the distribution of these columns to ensure the data is reasonable for further analysis.


In [44]:
# Count occurrences of zero values in volume, wholesale price, and retail price
zero_volume_count = combined_data[combined_data['volume_supplied_tonnes'] == 0].shape[0]
zero_wholesale_count = combined_data[combined_data['wholesale_price_ksh'] == 0].shape[0]
zero_retail_count = combined_data[combined_data['retail_price_ksh'] == 0].shape[0]

# Display the results
print(f"Rows with zero volume: {zero_volume_count}")
print(f"Rows with zero wholesale price: {zero_wholesale_count}")
print(f"Rows with zero retail price: {zero_retail_count}")


Rows with zero volume: 8020
Rows with zero wholesale price: 57
Rows with zero retail price: 3


In [45]:
# Step 10: Drop rows where volume, wholesale price, or retail price is zero

# Drop rows where volume_supplied_tonnes is 0
combined_data = combined_data[combined_data['volume_supplied_tonnes'] != 0]

# Drop rows where wholesale_price_ksh is 0
combined_data = combined_data[combined_data['wholesale_price_ksh'] != 0]

# Drop rows where retail_price_ksh is 0
combined_data = combined_data[combined_data['retail_price_ksh'] != 0]

# Re-inspect the dataset to confirm changes
print("\nData Summary After Dropping Zero Values:")
combined_data.info()

print("\nSummary Statistics After Dropping Zero Values:")
print(combined_data.describe())



Data Summary After Dropping Zero Values:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 975006 entries, 2023-10-17 to 2021-05-24
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   commodity               975006 non-null  object 
 1   classification          975006 non-null  object 
 2   county                  975006 non-null  object 
 3   market                  975006 non-null  object 
 4   volume_supplied_tonnes  975006 non-null  float64
 5   wholesale_price_ksh     975006 non-null  float64
 6   retail_price_ksh        975006 non-null  float64
dtypes: float64(3), object(4)
memory usage: 59.5+ MB

Summary Statistics After Dropping Zero Values:
       volume_supplied_tonnes  wholesale_price_ksh  retail_price_ksh
count           975006.000000         9.750060e+05      9.750060e+05
mean                 5.372975         2.564285e+03      2.919383e+03
std                361.334350         

### Calculating Outliers Using the IQR Method

In this step, I will calculate outliers for each commodity using the **Interquartile Range (IQR)** method. This approach is better suited for handling the variety of commodities in the dataset, as it accounts for differences in the price and volume distributions.

The IQR method defines outliers as values that fall below the lower bound or above the upper bound:
- Lower bound: Q1 - 1.5 * IQR
- Upper bound: Q3 + 1.5 * IQR

I will calculate the number of outliers in each of the following columns:
- `volume_supplied_tonnes`
- `wholesale_price_ksh`
- `retail_price_ksh`

This will be done on a **commodity-specific basis** to ensure that each commodity’s characteristics are considered when identifying outliers. I will summarize the results in a table to count the number of outliers for each commodity.


In [46]:
# Function to calculate IQR and count outliers for each commodity
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (df[column] < lower_bound) | (df[column] > upper_bound)

# Create a list to store the outlier summary data
outlier_summary_list = []

# Loop through each commodity and count the outliers
for commodity in combined_data['commodity'].unique():
    # Filter by commodity
    commodity_data = combined_data[combined_data['commodity'] == commodity]
    
    # Count outliers in volume, wholesale price, and retail price
    volume_outliers = detect_outliers_iqr(commodity_data, 'volume_supplied_tonnes').sum()
    wholesale_outliers = detect_outliers_iqr(commodity_data, 'wholesale_price_ksh').sum()
    retail_outliers = detect_outliers_iqr(commodity_data, 'retail_price_ksh').sum()
    
    # Append to the list
    outlier_summary_list.append({
        'commodity': commodity,
        'volume_outliers': volume_outliers,
        'wholesale_price_outliers': wholesale_outliers,
        'retail_price_outliers': retail_outliers
    })

# Convert the list into a DataFrame
outlier_summary = pd.DataFrame(outlier_summary_list)

# Display the summary of outliers
outlier_summary.sort_values(by=['volume_outliers', 'wholesale_price_outliers', 'retail_price_outliers'], ascending=False)


,commodity,volume_outliers,wholesale_price_outliers,retail_price_outliers
21,Cattle,4875,542,9320
55,Goat,4515,1104,1
151,Sheep,3805,832,5598
44,Eggs,2838,95,70
41,Dry Onions,2802,2543,4792
...,...,...,...,...
121,Rabbit Meat,0,0,1
9,Barbus,0,0,0
31,Cotton,0,0,0
40,commodity,0,0,0


### Dropping All Outliers Using the IQR Method

In this step, I will drop all outliers identified using the **Interquartile Range (IQR)** method. This is done on a commodity-specific basis, where outliers in the `volume_supplied_tonnes`, `wholesale_price_ksh`, and `retail_price_ksh` columns are removed. 

Since they are less than 0.02% of the dataset, the noise will skew the visual analysis and Machine Learning Model.

After dropping the outliers, I will re-inspect the dataset to confirm that it is clean and ready for further analysis.


In [48]:
# Create an empty list to store cleaned data for each commodity
cleaned_data_chunks = []

# Loop through each commodity and drop the outliers identified using the IQR method
for commodity in combined_data['commodity'].unique():
    # Filter the data for each commodity
    commodity_data = combined_data[combined_data['commodity'] == commodity].copy()
    
    # Identify outliers in volume, wholesale price, and retail price
    volume_outliers = detect_outliers_iqr(commodity_data, 'volume_supplied_tonnes')
    wholesale_outliers = detect_outliers_iqr(commodity_data, 'wholesale_price_ksh')
    retail_outliers = detect_outliers_iqr(commodity_data, 'retail_price_ksh')
    
    # Drop outliers
    commodity_cleaned_data = commodity_data[~(volume_outliers | wholesale_outliers | retail_outliers)]
    
    # Append cleaned commodity data to the list
    cleaned_data_chunks.append(commodity_cleaned_data)

# Concatenate the list of cleaned data back into a single DataFrame
combined_data_cleaned = pd.concat(cleaned_data_chunks)

# Re-inspect the dataset to confirm changes after dropping all outliers
print("\nUpdated dataset info after dropping all outliers:")
combined_data_cleaned.info()

# Recheck summary statistics after handling outliers
print("\nSummary Statistics After Dropping All Outliers:")
print(combined_data_cleaned.describe())



Updated dataset info after dropping all outliers:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 783418 entries, 2023-09-26 to 2021-05-24
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   commodity               783418 non-null  object 
 1   classification          783418 non-null  object 
 2   county                  783418 non-null  object 
 3   market                  783418 non-null  object 
 4   volume_supplied_tonnes  783418 non-null  float64
 5   wholesale_price_ksh     783418 non-null  float64
 6   retail_price_ksh        783418 non-null  float64
dtypes: float64(3), object(4)
memory usage: 47.8+ MB

Summary Statistics After Dropping All Outliers:
       volume_supplied_tonnes  wholesale_price_ksh  retail_price_ksh
count           783418.000000        783418.000000     783418.000000
mean                 1.452679          1642.937342       2906.764626
std                  2.96462

### Dropping Rows Where Retail Prices are Lower than Wholesale Prices

In this step, I will identify and drop rows where the `retail_price_ksh` is lower than the `wholesale_price_ksh`. This is a clear data entry error since retail prices should logically be higher than wholesale prices. These erroneous rows will be removed to ensure the integrity of the dataset.


In [49]:
# Step: Drop rows where retail price is lower than wholesale price
invalid_price_rows = combined_data_cleaned[combined_data_cleaned['retail_price_ksh'] < combined_data_cleaned['wholesale_price_ksh']]

# Display a sample of these rows for review
print("\nSample rows where retail price is lower than wholesale price:")
print(invalid_price_rows.head())

# Drop the invalid rows
combined_data_cleaned = combined_data_cleaned[combined_data_cleaned['retail_price_ksh'] >= combined_data_cleaned['wholesale_price_ksh']]

# Confirm that the rows have been dropped
print("\nUpdated dataset info after dropping invalid retail prices:")
combined_data_cleaned.info()

# Recheck summary statistics to ensure the dataset is still clean
print("\nSummary Statistics After Dropping Invalid Prices:")
print(combined_data_cleaned.describe())



Sample rows where retail price is lower than wholesale price:
            commodity classification        county                    market  \
date                                                                           
2021-07-07    Alestes          Dried         Nandi  Chepterit Market - Nandi   
2021-05-24  Anchovies              -         Kwale                  Mwangulu   
2024-09-15     Apples              -        Kisumu                    Kibuye   
2024-09-14     Apples              -       Nairobi          Nairobi Wakulima   
2024-09-06     Apples              -  Taita-Taveta                Voi Retail   

            volume_supplied_tonnes  wholesale_price_ksh  retail_price_ksh  
date                                                                       
2021-07-07                  0.1000              125.000             100.0  
2021-05-24                  0.2000              214.290             200.0  
2024-09-15                  0.1225              223.235             200.

---

### Final Inspection of the Cleaned Dataset

After completing the cleaning process by handling outliers and removing invalid rows where retail prices were lower than wholesale prices, I will now perform a final inspection of the dataset to ensure it is fully clean and ready for analysis.

The final checks will include checking for missing values: Ensuring no missing values remain in the dataset.



In [50]:
# Check for any remaining missing values in the dataset
missing_values = combined_data_cleaned.isnull().sum()
print("\nMissing Values After Cleaning:")
print(missing_values)



Missing Values After Cleaning:
commodity                 0
classification            0
county                    0
market                    0
volume_supplied_tonnes    0
wholesale_price_ksh       0
retail_price_ksh          0
dtype: int64


---

### Final Step: Splitting the Dataset into Categorical Datasets

In this step, I will split the combined dataset into separate datasets based on the predefined categories: **Seasonal Crops**, **Perennial Crops**, **Livestock**, **Meat**, **Animal Products**, **Animal By-products**, **Salt Water Fish**, **Fresh Water Fish**, and **Agricultural Inputs**. Any commodities that do not fit into these categories will be grouped into an "Other" category.

After splitting, I will inspect the first few rows of each categorical dataset to ensure they are correctly assigned.


In [62]:
# First, drop 'Pasta', 'Spaghetti', and any 'commodity' placeholders from the dataset
combined_data_cleaned = combined_data_cleaned[~combined_data_cleaned['commodity'].isin(['Pasta', 'Spaghetti', 'commodity'])]

# Strip leading and trailing spaces in the 'commodity' column
combined_data_cleaned['commodity'] = combined_data_cleaned['commodity'].str.strip()

# Define commodity categories as lists
seasonal_crops = ['Arrow Root', 'Beans (Canadian wonder)', 'Beans (Mwezi Moja)', 'Beans Rosecoco (Nyayo)', 
                  'Beans Red Haricot (Wairimu)', 'Black nightshade (Managu/ Osuga)', 'Broccoli', 'Butternuts',
                  'Cassava Chips (dry)', 'Cassava Fresh', 'Cauliflower', 'Chillies', 'Coriander (Dhania)', 
                  'Cowpeas', 'Cowpea leaves (Kunde)', 'Courgette', 'Dry Maize', 'Dry Onions', 'Dry Peas', 
                  'Egg plant (Brinjals)', 'Ethiopian Kales - Kanzira', 'French beans', 'Fresh Peas', 'Garlic', 
                  'Ginger', 'Green Grams', 'Green Maize', 'Ground Nuts', 'Indigenous Crotolaria (Mito/Miro)', 
                  'Jute Plant (Murenda)', 'Lentils', 'Lettuce', 'Maize Bran', 'Maize Flour', 'Mixed Beans', 
                  'Nderema- Vine Spinach', 'Okra (Lady\'s fingers or Gumbo)', 'Pawpaw', 'Pigeon peas', 'Pumpkin', 
                  'Pumpkin Leaves', 'Red Sorghum', 'Spinach', 'Spider flower (Saga)', 'Sunflower Cake', 'Sunflower Oil', 
                  'Sunflower Seeds', 'Sweet Potatoes', 'Thorn melon', 'Water Melon', 'Wheat', 'Wheat Bran', 
                  'Wheat Flour', 'White Irish Potatoes', 'Yam', 'Kales/Sukuma Wiki', 'Cucumber', 'Capsicums', 
                  'Amaranthus (Terere)', 'Spring Onions', 'Tangerine (Sandara)', 'Rice', 'Njugu Mawe', 'Soybean oil']

perennial_crops = ['Apples', 'Avocado', 'Banana (Cooking)', 'Banana (Plantain)', 'Banana (Ripening)', 'Grapes', 
                   'Lemons', 'Limes', 'Mangoes Exotic', 'Mangoes Local', 'Oranges', 'Passion Fruits', 'Pineapples', 
                   'Coffee', 'Cotton', 'Cotton Seed', 'Macademia Seed', 'Pepino melon', 'Tea', 'Tree tomato']

livestock = ['Camel', 'Cattle', 'Chicken', 'Donkey', 'Duck', 'Goat', 'Sheep']

meat = ['Camel meat', 'Meat Beef', 'Meat Broiler', 'Meat Chevon', 'Meat Indigenous Chicken', 'Meat Mutton', 
        'Pork', 'Rabbit Meat']

animal_products = ['Cow Milk (At collection point)', 'Cow Milk (Processed)', 'Goat Milk (At collection point)', 
                   'Goat Milk (Processed)', 'Camel Milk (At collection point)', 'Camel milk(Processed)', 'Eggs', 
                   'Honey', 'Fish Oil']

animal_byproducts = ['Camel Hide', 'Cattle Hide', 'Goat Skin', 'Sheep Skin', 'Fish Scales', 'Nile Perch Skins']

salt_water_fish = ['Anchovies', 'Barracuda(Kasumba)', 'Cuttlefish', 'Golden (Deep-Sea) Crabs Kaa', 'Goatfishes', 
                   'Groupers', 'Grunt(Taamamba/Kora)', 'Halfbeaks', 'Jobfish', 'Kingfish (Nguru)', 'Lobster(Kamba Mawe)', 
                   'Mackerel', 'Mud Crabs', 'Mullets(Fumi)', 'Octopus (Pweza)', 'Oysters', 'Parrotfishes(Pono)', 
                   'Queenfish (Pandu)', 'Rayfish', 'Rockcode(Tewa)', 'Sailfishes', 'Sardines', 'Scavengers (Changu/Tangu)', 
                   'Sharks', 'Snappers(Tazanda)', 'Surgeonfishes', 'Swordfishes', 'Threadfin breams', 'Tuna', 'Wolf Herrings', 
                   'Jacks/Trevallies(Kolekole)', 'Marlins', 'Prawns', 'Squid(Ngisi)']

fresh_water_fish = ['African butter catfish', 'Alestes', 'Barbus', 'Black bass', 'Common Carp', 'Fresh Water Shrimp', 
                    'Haplochromis', 'Mixed Demersal', 'Mixed Pelagics', 'Nile Perch', 'Protopterus', 'Synodontis', 
                    'Tilapia', 'Trout', 'Omena', 'Labeo', 'Mormyrus']

agricultural_inputs = ['Fertilizer']

# Create separate datasets
seasonal_crops_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(seasonal_crops)]
perennial_crops_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(perennial_crops)]
livestock_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(livestock)]
meat_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(meat)]
animal_products_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(animal_products)]
animal_byproducts_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(animal_byproducts)]
salt_water_fish_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(salt_water_fish)]
fresh_water_fish_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(fresh_water_fish)]
agricultural_inputs_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(agricultural_inputs)]

# Check for any commodities that don't fit into any category (Other category)
all_categorized_commodities = (seasonal_crops + perennial_crops + livestock + meat + animal_products + 
                               animal_byproducts + salt_water_fish + fresh_water_fish + agricultural_inputs)

other_data = combined_data_cleaned[~combined_data_cleaned['commodity'].isin(all_categorized_commodities)]

# Return unique commodities in each dataset
print("Unique commodities in Seasonal Crops Data:")
print(seasonal_crops_data['commodity'].unique(), "\n")

print("Unique commodities in Perennial Crops Data:")
print(perennial_crops_data['commodity'].unique(), "\n")

print("Unique commodities in Livestock Data:")
print(livestock_data['commodity'].unique(), "\n")

print("Unique commodities in Meat Data:")
print(meat_data['commodity'].unique(), "\n")

print("Unique commodities in Animal Products Data:")
print(animal_products_data['commodity'].unique(), "\n")

print("Unique commodities in Animal By-products Data:")
print(animal_byproducts_data['commodity'].unique(), "\n")

print("Unique commodities in Salt Water Fish Data:")
print(salt_water_fish_data['commodity'].unique(), "\n")

print("Unique commodities in Fresh Water Fish Data:")
print(fresh_water_fish_data['commodity'].unique(), "\n")

print("Unique commodities in Agricultural Inputs Data:")
print(agricultural_inputs_data['commodity'].unique(), "\n")

print("Unique commodities in Other Data (if any):")
print(other_data['commodity'].unique(), "\n")


Unique commodities in Seasonal Crops Data:
['Arrow Root' 'Beans (Canadian wonder)' 'Beans (Mwezi Moja)' 'Broccoli'
 'Butternuts' 'Cassava Chips (dry)' 'Cassava Fresh' 'Cauliflower'
 'Chillies' 'Coriander (Dhania)' 'Courgette' 'Cowpeas'
 'Cowpea leaves (Kunde)' 'Dry Maize' 'Dry Onions' 'Dry Peas'
 'Egg plant (Brinjals)' 'French beans' 'Fresh Peas' 'Garlic' 'Ginger'
 'Green Grams' 'Green Maize' 'Ground Nuts'
 'Indigenous Crotolaria (Mito/Miro)' 'Lentils' 'Lettuce' 'Maize Bran'
 'Maize Flour' 'Mixed Beans' 'Nderema- Vine Spinach' 'Njugu Mawe'
 "Okra (Lady's fingers or Gumbo)" 'Pawpaw' 'Pigeon peas' 'Pumpkin'
 'Pumpkin Leaves' 'Red Sorghum' 'Jute Plant (Murenda)'
 'Beans Red Haricot (Wairimu)' 'Black nightshade (Managu/ Osuga)'
 'Spider flower (Saga)' 'Amaranthus (Terere)' 'Yam' 'Thorn melon'
 'Kales/Sukuma Wiki' 'Cucumber' 'Beans Rosecoco (Nyayo)' 'Spinach'
 'Capsicums' 'Rice' 'Soybean oil' 'Spring Onions' 'Sunflower Cake'
 'Sunflower Oil' 'Sunflower Seeds' 'Tangerine (Sandara)' 'Water Me

### Identified Data Entry Issues and the Way Forward

After inspecting the unique commodities in the "Other Data" category, it is clear that several commodities are incorrectly classified due to minor typing inconsistencies or formatting issues. For example, items like `'Cow Milk(Processd)'` and `'Goat milk (Processed)'` have inconsistent spacing, capitalization, or punctuation.

**Way Forward**:
1. **Fix Typing Issues**: I will normalize these commodity names by correcting the inconsistent names using replacements.
2. **Reassign Commodities**: After correcting the typing errors, I will reassign these commodities to their appropriate categories.
3. **Update Categories**: Once the commodities are correctly assigned, they will be appended to their respective categorical datasets.

In [63]:
# Step 1: Fix Typing Issues in the 'commodity' Column
corrections = {
    'Cow Milk(Processd)': 'Cow Milk (Processed)',
    'Goat milk (Processed)': 'Goat Milk (Processed)',
    'Goat Milk (at collection point)': 'Goat Milk (At collection point)',
    'Mangoes': 'Mangoes Exotic',  # Assuming this refers to exotic mangoes
    'Meat Indiginous Chicken': 'Meat Indigenous Chicken',  # Correct the spelling
    'Camel Milk(At collection point)': 'Camel Milk (At collection point)',
    'Cow Milk(At collection point)': 'Cow Milk (At collection point)',
    'Ethiopian Kales -Kanzira': 'Ethiopian Kales - Kanzira',
    'Needlefishes': 'Needlefishes',  # Assuming this is correct
    'Soybean oil': 'Soybean oil'  # Move this to Seasonal Crops as requested
}

# Apply corrections to the 'commodity' column
combined_data_cleaned['commodity'] = combined_data_cleaned['commodity'].replace(corrections)

# Step 2: Reassign corrected commodities to their respective categories
# Reassign these commodities after correcting typing issues

# Update the seasonal crops category to include 'Soybean oil'
seasonal_crops.append('Soybean oil')

# Recreate the datasets after fixing the typing issues
seasonal_crops_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(seasonal_crops)]
perennial_crops_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(perennial_crops)]
livestock_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(livestock)]
meat_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(meat)]
animal_products_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(animal_products)]
animal_byproducts_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(animal_byproducts)]
salt_water_fish_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(salt_water_fish)]
fresh_water_fish_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(fresh_water_fish)]
agricultural_inputs_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(agricultural_inputs)]

# Step 3: Check for any remaining commodities that don't fit into any category (Other category)
all_categorized_commodities = (seasonal_crops + perennial_crops + livestock + meat + animal_products + 
                               animal_byproducts + salt_water_fish + fresh_water_fish + agricultural_inputs)

other_data = combined_data_cleaned[~combined_data_cleaned['commodity'].isin(all_categorized_commodities)]

# Return unique commodities in Other Data again to check if everything has been properly categorized
print("Unique commodities in Other Data (if any):")
print(other_data['commodity'].unique(), "\n")


Unique commodities in Other Data (if any):
['Coconut' 'Coconut Oil' 'Needlefishes' 'Other Fresh Water' 'Paddy'
 'Fish Maws'] 



In [64]:
# Step 1: Correctly assign the remaining commodities to their respective categories
# Add 'Coconut' and 'Coconut Oil' to Perennial Crops
perennial_crops += ['Coconut', 'Coconut Oil']

# Add 'Other Fresh Water' and 'Fish Maws' to Fresh Water Fish
fresh_water_fish += ['Other Fresh Water', 'Fish Maws']

# Add 'Paddy' to Seasonal Crops (since it's similar to rice)
seasonal_crops.append('Paddy')

# Add 'Needlefishes' to Salt Water Fish
salt_water_fish.append('Needlefishes')

# Step 2: Recreate the datasets with the updated category lists
seasonal_crops_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(seasonal_crops)]
perennial_crops_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(perennial_crops)]
livestock_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(livestock)]
meat_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(meat)]
animal_products_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(animal_products)]
animal_byproducts_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(animal_byproducts)]
salt_water_fish_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(salt_water_fish)]
fresh_water_fish_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(fresh_water_fish)]
agricultural_inputs_data = combined_data_cleaned[combined_data_cleaned['commodity'].isin(agricultural_inputs)]

# Step 3: Check if there are any remaining commodities in the "Other" category
all_categorized_commodities = (seasonal_crops + perennial_crops + livestock + meat + animal_products + 
                               animal_byproducts + salt_water_fish + fresh_water_fish + agricultural_inputs)

other_data = combined_data_cleaned[~combined_data_cleaned['commodity'].isin(all_categorized_commodities)]

# Step 4: Inspect the "Other Data" to ensure all commodities are correctly assigned
print("Unique commodities in Other Data (if any):")
print(other_data['commodity'].unique(), "\n")


Unique commodities in Other Data (if any):
[] 



---

### Exporting the Combined Dataset and Categorical Datasets into CSV Files

In this step, I will export the combined dataset and all individual category datasets into CSV files. These CSV files will be used for further analysis in separate notebooks, where exploratory data analysis (EDA) and preprocessing will be conducted for each category before moving toward predictive model building for supply volumes and market prices.



In [65]:
# Export the combined dataset to CSV
combined_data_cleaned.to_csv('combined_dataset_cleaned.csv', index=True)

# Export the individual categorical datasets to CSV
seasonal_crops_data.to_csv('seasonal_crops_data.csv', index=True)
perennial_crops_data.to_csv('perennial_crops_data.csv', index=True)
livestock_data.to_csv('livestock_data.csv', index=True)
meat_data.to_csv('meat_data.csv', index=True)
animal_products_data.to_csv('animal_products_data.csv', index=True)
animal_byproducts_data.to_csv('animal_byproducts_data.csv', index=True)
salt_water_fish_data.to_csv('salt_water_fish_data.csv', index=True)
fresh_water_fish_data.to_csv('fresh_water_fish_data.csv', index=True)
agricultural_inputs_data.to_csv('agricultural_inputs_data.csv', index=True)

# Check if any commodities were not categorized (Other category)
if not other_data.empty:
    other_data.to_csv('other_data.csv', index=True)


---

### Conclusion and Areas of Further Analysis

This project has successfully cleaned, processed, and categorized the agricultural dataset into meaningful groups, allowing for better analysis and insight extraction. The data has been categorized into **Seasonal Crops**, **Perennial Crops**, **Livestock**, **Meat**, **Animal Products**, **Animal By-products**, **Salt Water Fish**, **Fresh Water Fish**, and **Agricultural Inputs**. 

The next steps will involve conducting **Exploratory Data Analysis (EDA)** for each category in separate notebooks to further investigate trends in pricing and supply volumes. The aim will be to identify patterns in the data, including **seasonal trends**, **geographical supply trends**, and **price fluctuations** across different commodities and markets.

Following EDA, the focus will be on **building predictive models** to forecast:
1. **Supply Volumes**: Using time series and regression analysis to predict how supply volumes vary over time and across regions.
2. **Market Prices**: Developing predictive models to forecast wholesale and retail prices, enabling stakeholders to make data-driven decisions.

Key areas of exploration include:
- **Seasonality and Cyclical Patterns**: Identifying seasonal patterns in supply volumes and pricing for different commodities.
- **Geographical Analysis**: Investigating how supply and demand trends differ across regions and markets.
- **Market Linkage and Optimization**: Exploring how market players can better align their supply chains to maximize profitability and efficiency.

With this data, we can aim to build robust models that provide actionable insights for farmers, market players, and other stakeholders involved in the agricultural value chain.

---



---

© 2024 All rights reserved. 

Prepared by: Edward Njiru  
Project: Agricultural Commodity Supply and Pricing Analysis  
Date: September 2024  

---
